<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/DCGAN_Pytorch_implementantion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>>You are in CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>>{type(e)}: {e}\n>>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.716)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h} hrs: {m:>02} min: {s:>05.2f} sec"
print(f">>>>time formating:\tplease wait....\n>>>>time elapsed:\t{time_fmt()}")

Mounted at /content/drive
>>>>You are in CoLaB with torch version: 1.8.1+cu101
>>>>time formating:	please wait....
>>>>time elapsed:	0 hrs: 02 min: 03.00 sec


In [31]:
#In this notbook we will train a modified simple GAN using CNN -we implement the DCGAN

In [32]:
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import time, math, random
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [33]:
#Set the seed for reproducability and gpu to deterministic to avoid errors during training:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [34]:
#This Model consists of two main part. The generator (to produce fake data) with main
# aim of fooling the descriminator. The discriminator to classify fake data from real data

In [35]:
class Discriminator(nn.Module):
  def __init__(self, img_channels, d_features):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
        nn.Conv2d(img_channels, d_features, kernel_size = 4, stride = 2, padding = 1),
        nn.LeakyReLU(0.2),
        self._disc_block(d_features, 2 * d_features, 4, 2, 1),
        self._disc_block(2 * d_features, 4 * d_features, 4, 2, 1),
        self._disc_block(4 * d_features, 8 * d_features, 4, 2, 1),
        nn.Conv2d(8 * d_features, 1,kernel_size = 4, stride = 2, padding = 0),
        nn.Sigmoid())
    
  def _disc_block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias = False),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2))
  
  def forward(self, input_tensor):
    return self.discriminator(input_tensor)

In [36]:
class Generator(nn.Module):
  def __init__(self, z_dim, in_channels, g_features):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        self._build(z_dim, g_features * 16, 4, 2, 0), #shape = [4,4]
        self._build(g_features * 16, 8*g_features, 4,2,1),#shape = [8,8]
        self._build(8 * g_features, 4 * g_features, 4, 2, 1),#shape = [16,16]
        self._build(4 * g_features, 2 * g_features, 4, 2, 1), # shape = [32,32]
        nn.ConvTranspose2d(2 * g_features, in_channels, kernel_size = 4, stride = 2, padding =1),#shape = [64,64]
        nn.Tanh())
    
  def _build(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU())
    
  def forward(self, input_tensor):
    return self.generator(input_tensor)


In [37]:
#Initialize the weights for the model to random normal:
def wt_initializer(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, mean = 0.00, std = 0.001)


In [38]:
#We can initialize and testing the model class to see if it works as we expected:
def test():
  z_dim = 100
  img_channels = 3
  H = 64
  W = 64
  batch_size = 8
  fake = torch.randn((batch_size,img_channels, H, W))
  noise = torch.randn((batch_size, z_dim, 1,1))
  discriminator = Discriminator(img_channels,8)
  generator = Generator(z_dim, img_channels, 8)
  wt_initializer(discriminator)
  assert discriminator(fake).shape == (batch_size, 1,1,1)
  wt_initializer(generator)
  assert generator(noise).shape == (batch_size, img_channels, H, W)
  return generator(noise).shape, discriminator(fake).shape

In [39]:
test()

(torch.Size([8, 3, 64, 64]), torch.Size([8, 1, 1, 1]))

In [40]:
#Prepare the data from torchvision
batch_size = 64
img_channels = 1
img_size = 64
gen_features = 64
disc_features = 64
EPOCHS = 5
learning_rate = 2e-4
z_dim = 100
transforms = transforms.Compose([transforms.ToTensor(), 
                                 transforms.Normalize([0.5 for _ in range(img_channels)],[0.5 for _ in range(img_channels)]), 
                                 transforms.Resize(img_size)])
dfm = datasets.MNIST(root = "mnist/", transform = transforms, download = True,)
loader = DataLoader(dfm, batch_size = batch_size, shuffle = True)
x_dfm, y_dfm = next(iter(loader))
print(f"x_dfm_shape: {x_dfm.shape}\ty_dfm_shape: {y_dfm.shape}")

x_dfm_shape: torch.Size([64, 1, 64, 64])	y_dfm_shape: torch.Size([64])


In [41]:
#Get the optimizer and the loss object for the training
generator = Generator(z_dim, img_channels, gen_features).to(device = device)
discriminator = Discriminator(img_channels, disc_features).to(device = device)
fixed_noise = torch.randn((32, z_dim, 1,1)).to(device = device)
gen_optimizer = optim.Adam(params = generator.parameters(), lr = learning_rate, betas = (0.5, 0.999))
disc_optimizer = optim.Adam(params = discriminator.parameters(), lr = learning_rate, betas = (0.5, 0.999))
loss_obj = nn.BCELoss()


In [42]:
#The training loop:
step = 0
writer1 = SummaryWriter(f"logs/real_data")
writer2 = SummaryWriter(f"logs/fake_data")
tic = time.time()
for epoch in range(EPOCHS):
  tic1 = time.time()
  print(f"\n>>>>training start for epoch: {epoch + 1}\tplease wait......")
  for idx, (real,_) in enumerate(tqdm(loader)):
    real = real.to(device = device)
    noise = torch.randn((batch_size, z_dim, 1,1)).to(device = device)
    fake = generator(noise)
    #To train discriminator we maximize [log(D(x)) + log(1 - D(G(z)))]
    disc_real_out = discriminator(real).reshape(-1)
    disc_real_loss = loss_obj(disc_real_out, torch.ones_like(disc_real_out))
    disc_fake_out = discriminator(fake).reshape(-1)
    disc_fake_loss = loss_obj(disc_fake_out, torch.zeros_like(disc_fake_out))
    disc_loss = (disc_real_loss + disc_fake_loss) / 2
    discriminator.zero_grad()
    disc_loss.backward(retain_graph = True)
    disc_optimizer.step()
    #train the generator we maximize [log(D(G(z)))]
    gen_out = discriminator(fake).reshape(-1)
    gen_loss = loss_obj(gen_out, torch.ones_like(gen_out))
    generator.zero_grad()
    gen_loss.backward()
    gen_optimizer.step()
    toc1 = time.time()
    if idx % 100 == 0:
      print(f"\n>>>>Epoch: {epoch + 1}/{EPOCHS}\ttime at the end of this epoch:{time_fmt(toc1 - tic1)} ")
      print(f">>>>Generator Loss: {gen_loss:.4f} | Generator PPL: {math.exp(gen_loss):7.4f}")
      print(f">>>>Discriminator Loss: {disc_loss:.4f} | Discriminator PPL: {math.exp(disc_loss):7.4f}")
      with torch.no_grad():
        fake = generator(fixed_noise)
        img_grid_real = torchvision.utils.make_grid(real[:32], normalize = True)
        img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize = True)
        writer1.add_image('Real_images', img_grid_real, global_step = step)
        writer2.add_image('Fake_images', img_grid_fake, global_step = step)
      step+=1
toc = time.time()
print(f"\n>>>>Total time elapsed to train this model is: {time_fmt(toc - tic)}")
    





  0%|          | 0/938 [00:00<?, ?it/s]


>>>>training start for epoch: 1	please wait......



>>>>Epoch: 1/5	time at the end of this epoch:0 hrs: 00 min: 00.00 sec 




  0%|          | 1/938 [00:00<06:29,  2.40it/s]

>>>>Generator Loss: 2.0245 | Generator PPL:  7.5721
>>>>Discriminator Loss: 0.6903 | Discriminator PPL:  1.9944


...


>>>>Epoch: 5/5	time at the end of this epoch:0 hrs: 02 min: 33.00 sec 
>>>>Generator Loss: 0.0000 | Generator PPL:  1.0000
>>>>Discriminator Loss: 50.0000 | Discriminator PPL: 5184705528587072045056.0000




 85%|████████▌ | 801/938 [02:33<00:37,  3.66it/s]

 86%|████████▌ | 803/938 [02:33<00:31,  4.34it/s]

 86%|████████▌ | 804/938 [02:33<00:29,  4.59it/s]

 86%|████████▌ | 805/938 [02:34<00:27,  4.77it/s]

 86%|████████▌ | 806/938 [02:34<00:26,  4.90it/s]

 86%|████████▌ | 807/938 [02:34<00:26,  5.01it/s]

 86%|████████▌ | 808/938 [02:34<00:25,  5.10it/s]

 86%|████████▌ | 809/938 [02:34<00:25,  5.15it/s]

 86%|████████▋ | 810/938 [02:34<00:24,  5.19it/s]

 86%|████████▋ | 811/938 [02:35<00:24,  5.20it/s]

 87%|████████▋ | 812/938 [02:35<00:24,  5.22it/s]

 87%|████████▋ | 813/938 [02:35<00:23,  5.22it/s]

 87%|████████▋ | 814/938 [02:35<00:23,  5.25it/s]

 87%|████████▋ | 815/938 [02:35<00:23,  5.25it/s]

 87%|████████▋ | 816/938 [02:36<00:23,  5.26it/s]

 87%|████████▋ | 817/938 [02:36<00:22,  5.28it/s]

 87%|████████▋ | 818/938 [02:36<00:22,  5.28it/s]

 87%|████████▋ | 819/938 [02:36<00:22,  5.25it/s]

 87%|████████▋ | 820/938 [02:36<00:22,  5.29it/s]

 88%|████████▊ | 821/938 [02:


>>>>Epoch: 5/5	time at the end of this epoch:0 hrs: 02 min: 52.00 sec 
>>>>Generator Loss: 0.0000 | Generator PPL:  1.0000
>>>>Discriminator Loss: 50.0000 | Discriminator PPL: 5184705528587072045056.0000




 96%|█████████▌| 901/938 [02:52<00:10,  3.64it/s]

 96%|█████████▋| 903/938 [02:52<00:08,  4.32it/s]

 96%|█████████▋| 904/938 [02:52<00:07,  4.55it/s]

 96%|█████████▋| 905/938 [02:53<00:06,  4.76it/s]

 97%|█████████▋| 906/938 [02:53<00:06,  4.89it/s]

 97%|█████████▋| 907/938 [02:53<00:06,  5.02it/s]

 97%|█████████▋| 908/938 [02:53<00:05,  5.10it/s]

 97%|█████████▋| 909/938 [02:53<00:05,  5.13it/s]

 97%|█████████▋| 910/938 [02:54<00:05,  5.18it/s]

 97%|█████████▋| 911/938 [02:54<00:05,  5.21it/s]

 97%|█████████▋| 912/938 [02:54<00:04,  5.23it/s]

 97%|█████████▋| 913/938 [02:54<00:04,  5.27it/s]

 97%|█████████▋| 914/938 [02:54<00:04,  5.24it/s]

 98%|█████████▊| 915/938 [02:55<00:04,  5.26it/s]

 98%|█████████▊| 916/938 [02:55<00:04,  5.29it/s]

 98%|█████████▊| 917/938 [02:55<00:03,  5.28it/s]

 98%|█████████▊| 918/938 [02:55<00:03,  5.26it/s]

 98%|█████████▊| 919/938 [02:55<00:03,  5.29it/s]

 98%|█████████▊| 920/938 [02:56<00:03,  5.28it/s]

 98%|█████████▊| 921/938 [02:


>>>>Total time elapsed to train this model is: 0 hrs: 15 min: 44.00 sec


In [43]:
!tensorboard --logdir=logs


2021-05-18 22:33:57.947928: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C
